# 1. Read NeXus NXDL files to extract fields and properties
# 2. Create ontology using owlready2

# To do:

-3. check nexus_class excel - check out ones with no superclass

look at NXxeuler:NXdetector

this is an interesting case as NXdetector is inside NXinstrument, inside NXxeuler, which extends NXxbase

>>> need to think again!!!

>>> have base classes inside base classes, base classes that extent base classes, base classes in definitions that extend base classes, base classes inside base classes in definitions ....

>> sketch out classes ...



For each class give complete class chain





-2. figure out how to determine preferred field name

-1. Is default type NXchar?? yes looks like it from nxdl.xds - done!

0. look through some base classes/app defs in spreadsheet and compare to manual (inc. NXaperture, NXsource)


1. separate unique names for each field; each has properties minOccurs, maxOccurs etc

2. Determine preferred unique field name for each

5. use dict for ontology

8. check: how to deal with groups inside classes/definitions - do they exist?

9. Read enumeration and add options to the ontology (enumeration is child of field or attribute - P134)



In [2]:
import xml.dom.minidom
import pandas as pd
import os
pd.set_option('display.max_rows',9999)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width',999)

In [23]:
'''
This section loads all NeXus NXDL files and creates a summary of all important parameters for each field 
    and each definition, separately, as lists of dictionaries.
    
Long unique class names are suggested.
Long unique field names are suggested.

A 'superclass' name is suggested, which is a generalizarion of 'extends' that includes implicit subclassing 
    inside application definitions

Caveats:
    The NeXus field names are assumes to have a specific meaning within the class in which they are defined. 
    However, some special classes such as NXtransformations (and formerly NXgeometry) refer instead to the 
    classes that contain these classes (e.g. NXgeometry in NXaperture refers to NXaperture's own geometry).
    Special cases should be made of these classes so they are connected to the parent classes.

The list of field dictionaries is converted to a pandas DataFrame and saved as an excel spreadsheet for convenience.

To do: 

check all superclasses - do the classes referred to exist and are they the correct ones?

generate 'Preferred' property - when a field is defined multiple times in the class hierarchy, the first occurence is preferred
This can be used to reduce unnecessary proliferation of unique field names.
'''


folder_pattern_list = ['/dls/science/users/spc93/misc_nexus_data/nexus_classes/baseclasses/NX*.xml',
          '/dls/science/users/spc93/misc_nexus_data/nexus_classes/definitions/NX*.xml']

outfile_definitions = '/home/spc93/ontology/nexus_definitions.xls'
outfile_fields = '/home/spc93/ontology/nexus_fields.xls'
outfile_classes = '/home/spc93/ontology/nexus_classes.xls'

join_string = ':'      #string added between joined names

def get_file_list_from_folder_list(folder_pattern_list):
    file_list = []
    for folder in folder_pattern_list:
        ff = os.popen('ls ' + folder)
        file_list += ff.read().splitlines()
    return(file_list)

        
file_list = get_file_list_from_folder_list(folder_pattern_list)

#defn_list = []         #list of dictionaries describing each nexus definition (one per xml file)
field_list = []         # list of dictionaries describing each nexus field (several per definition)

defn_dic = {} ### check this ise used

for file in file_list:
    #print(file)

    dom1 = xml.dom.minidom.parse(file)
      
    defn = dom1.getElementsByTagName('definition')[0]
    defn_name = defn.getAttribute('name') #class name from name attribute in definition
    category = defn.getAttribute('category')
    extends = defn.getAttribute('extends')
    class_doc = dom1.getElementsByTagName('doc')[0].firstChild.nodeValue.replace('\n','')
    
    #defn_list += [{
    #    'defn_name' : defn_name,
    #    'xml_file' : file.split('/')[-1],
    #    'category' : category,
    #    'extends' : extends,
    #    'doc' : class_doc
    #    }]
    

    defn_dic[defn_name] = {
        'xml_file' : file.split('/')[-1],
        'category' : category,
        'extends' : extends,
        'doc' : class_doc      
    }
   
    for field in defn.getElementsByTagName('field'):
        field_name = field.getAttribute('name')
        units = field.getAttribute('units')
        _type = field.getAttribute('type')
        if _type == '':
            _type = 'NX_CHAR'   # default if not specified
        
        parent_type = field.parentNode.getAttribute('type') # will  be NXentry if top level of definition, then replace with empty string?
        
        # the following logic defines the unique long class names
        # for fields in application, append definition name to make a new unique class name
        if category == 'base':
            long_class_name = defn_name
        elif category == 'application':
            if parent_type == 'NXentry':
                long_class_name = defn_name                
            else:
                long_class_name = '%s%s%s' % (defn_name, join_string, parent_type)
                
                
        # the following defines a superclass for each class (long_class_name)
        # it is a generalization of 'extends' that includes implicit subclassing inside application definitions
        #if category == 'application':
        #    if extends == 'NXobject':
        #        superclass = parent_type
        #    else:
        #        superclass = '%s%s%s' % (extends, join_string, parent_type)    
        #elif category == 'base':
        #    superclass = extends
 

        try:
            field_doc = field.getElementsByTagName('doc')[0].firstChild.nodeValue.replace('\n','')
        except:
            field_doc = ''
            
        long_field_name = '%s%s%s' % (long_class_name, join_string, field_name)

        field_list += [{
            'field_name' : field_name,
            'parent_type' : parent_type,
            'long_class_name' : long_class_name,  
            'long_field_name' : long_field_name,
            'defn_name' : defn_name,          
            'category' : category,
            'extends' : extends,
            #'superclass' : superclass,
            'units' : units,
            'type' : _type,
            'xml_file' : file.split('/')[-1],
            'doc' : field_doc
            }]
        

        

#definitions = pd.DataFrame(defn_list)
definitions = pd.DataFrame(defn_dic).transpose()

fields = pd.DataFrame(field_list)

definitions.to_excel(outfile_definitions)
fields.to_excel(outfile_fields)

definitions
#fields



,xml_file,category,extends,doc
NXaperture,NXaperture.nxdl.xml,base,NXobject,A beamline aperture.
NXattenuator,NXattenuator.nxdl.xml,base,NXobject,A device that reduces the intensity of...
NXbeam,NXbeam.nxdl.xml,base,NXobject,Properties of the neutron or X-ray bea...
NXbeam_stop,NXbeam_stop.nxdl.xml,base,NXobject,A device that blocks the beam complete...
NXbending_magnet,NXbending_magnet.nxdl.xml,base,NXobject,A bending magnet
NXcapillary,NXcapillary.nxdl.xml,base,NXobject,A capillary lens to focus the X-ray beam...
NXcite,NXcite.nxdl.xml,base,NXobject,\t\tA literature reference\t\t\t\tDefinition t...
NXcollection,NXcollection.nxdl.xml,base,NXobject,"\t\tAn unvalidated set of terms, such as the d..."
NXcollimator,NXcollimator.nxdl.xml,base,NXobject,A beamline collimator.
NXcrystal,NXcrystal.nxdl.xml,base,NXobject,These symbols will be used below to coordinate...


In [28]:
defn_dic['NXxlaue']['extends']
defn_dic['NXxrot']['extends']
defn_dic['NXxbase']['extends']
#pd.DataFrame(defn_dic).transpose()

'NXobject'

###########  abandon this one #####

def find_class_chain(field_name, chain_list, field_present_list): 

    idx = chain_list[-1]
    
    field_name = field_list[idx]['field_name']
    superclass = field_list[idx]['superclass']
    
    print('field_name: ', field_name, 'superclass: ', superclass)
       
    if superclass == 'NXobject':
        return (field_name, chain_list, field_present_list)
    #Find next superclass (working backwards)
    else:
        _found_class = 0
        
        
        for next_idx in range(len(field_list)):
            
            if field_list[next_idx]['field_name'] == field_name and field_list[i]['long_class_name'] == superclass:
                _found_class = 1
                break
                
        print(field_name, chain_list + [next_idx], field_present_list + [_found_class])
        
        find_class_chain(field_name, chain_list + [next_idx], field_present_list + [_found_class])


find_class_chain('y_translation', [1156], [1])


In [55]:


# create a list of all unique classes (based on long class names), with fields required for determining superclass
class_dict = {}

max_depth = 5

for field in field_list:
    class_dict[field['long_class_name']] = {'category': field['category'],
                                            'extends': field['extends'],
                                            'parent_type': field['parent_type']
                                           }

for long_class_name in class_dict.keys(): 
    category, extends, parent_type = class_dict[long_class_name]['category'], class_dict[long_class_name]['extends'], class_dict[long_class_name]['parent_type'] 
#for field in field_list:
    
    #long_class_name, category, extends, parent_type = field['long_class_name'], field['category'], field['extends'], field['parent_type']

    if category == 'base': #for base classes the superclass is simply given by 'extends'
        superclass = extends
        
    elif category == 'application': #for application definitions there is an implied superclass of class inside to definition that is extended by the current one
        if extends == 'NXobject':
            superclass = parent_type
        else:
            superclass = '%s%s%s' % (extends, join_string, parent_type)        
            if superclass in class_dict:
                break

            for i in range(max_depth):
                defn = defn_dic[extends]['extends'] # look for definition that extends the superclass we are looking for
                if defn == 'NXobject':
                    superclass = parent_type
                    break
                else:
                    superclass = '%s%s%s' % (defn, join_string, parent_type)  
                    if superclass in class_dict:
                        break
                    else:
                        superclass = ''


    #field['superclass'] = superclass
    class_dict[long_class_name]['superclass'] = superclass

    
#fields = pd.DataFrame(field_list)

#fields.to_excel(outfile_fields)
    
outfile_classes = '/home/spc93/ontology/nexus_classes.xls'
classes = pd.DataFrame(class_dict).transpose()
classes.to_excel(outfile_classes)


In [49]:
defn_dic['NXsample_component']['extends']
    

'NXobject'

{'NXaperture': {'category': 'base'},
 'NXattenuator': {'category': 'base'},
 'NXbeam': {'category': 'base'},
 'NXbeam_stop': {'category': 'base'},
 'NXbending_magnet': {'category': 'base'},
 'NXcapillary': {'category': 'base'},
 'NXcite': {'category': 'base'},
 'NXcollimator': {'category': 'base'},
 'NXcrystal': {'category': 'base'},
 'NXcylindrical_geometry': {'category': 'base'},
 'NXdata': {'category': 'base'},
 'NXdetector': {'category': 'base'},
 'NXdetector_group': {'category': 'base'},
 'NXdetector_module': {'category': 'base'},
 'NXdisk_chopper': {'category': 'base'},
 'NXentry': {'category': 'base'},
 'NXenvironment': {'category': 'base'},
 'NXevent_data': {'category': 'base'},
 'NXfermi_chopper': {'category': 'base'},
 'NXfilter': {'category': 'base'},
 'NXflipper': {'category': 'base'},
 'NXfresnel_zone_plate': {'category': 'base'},
 'NXgeometry': {'category': 'base'},
 'NXgrating': {'category': 'base'},
 'NXguide': {'category': 'base'},
 'NXinsertion_device': {'category': '

In [89]:
sort(a)

NameError: name 'sort' is not defined

In [64]:
print(len(class_set), len(superclass_set))
superclass_set.issubset(class_set)
superclass_set.difference(class_set) # superclasses that are not classes (should be none!)

231 35


{'NXcollection',
 'NXtofraw:NXdisk_chopper',
 'NXtofraw:NXentry',
 'NXtofraw:NXfermi_chopper',
 'NXtofraw:NXmonochromator',
 'NXxbase:NXattenuator',
 'NXxbase:NXentry',
 'NXxlaue:NXdetector',
 'NXxlaue:NXentry',
 'NXxrot:NXdata',
 'NXxrot:NXentry'}

In [33]:
for i in range(len(field_list)):
    if field_list[i]['field_name'] == 'temperature':
        break
print(i)




95


In [15]:
field_list[10]['field_name'] == 'temperature'

False

In [29]:
[i for i in field_list if field_list[i]['field_name'] == 'temperature']

TypeError: list indices must be integers or slices, not dict

In [ ]:
matches = [x for x in lst if fulfills_some_condition(x)]
matches = (x for x in lst if x > 6)

In [27]:
#field_list[4]['field_name'] == 'temperature'
next((i for i, item in enumerate(dicts) if item["name"] == "Pam"), None)

False

In [21]:
#pip install owlready2

from owlready2 import *
#onto_path.append("/dls/science/users/spc93/ontology")

onto = get_ontology('file:///home/spc93/ontology/NeXus_onto1.owl')



Must use long names for NeXus fields as the names need to be unique in the ontology namespace
Property restrictions not implemented - are they needed? how to make anonymous class?
Treat units as classes with superclass Unit Category?? (or individuals?)

NeXus ADs have fields that sometimes have minOccurs attribute (string)
Defaults (if not set): 0 (base classes); 1 (definitions)

If extended other class or in definition (extended base class) then subclass gives only new fields. Existing fields are given implicitly.

??? The ontology cannot then give a new value for minOccurs so minOccurs can't be in the ontology??? - this needs some thought

Limitations of the ontology:
1. Need long names to avoid name clashes
2. Can't have minOccurs as they belong to the short field names
No - can have all fields (with long names) but set 'uniqueField' to True or False (use only unique fields for catalogue names). Assume extends is a subclassing mechanism with no overload. (Also implies minOccurs >= superclass version).

Need to assume field names are unique in 'namespace' (base class or definition)


In [22]:
# seems not to require namespace if using 'with'


with onto:

    ## Classes (generic)

    class NeXus(Thing):
        comment = 'NeXus concept bla bla'
    
    class NeXusField(NeXus):
        comment = 'NeXus field. Unique names are created by prepending the NeXus class name to the NeXus field name'

    class NXobject(NeXus):
        comment = 'NXobject'
    
    class NeXusBaseClass(NXobject):
        comment = 'NeXusBaseClass'
    
    class NeXusDefinition(NXobject):
        comment = 'NeXus Application Definition'
        
    class NeXusUnitCategory(NeXus):
        comment = 'NeXus Unit Category'
        
    class NeXusUnit(NeXusUnitCategory):
        comment = 'NeXus Unit'
        
    ## properties (generic)
    
    class hasField(NXobject >> NeXusField):
        comment = 'NXobject has NeXus field relationship'
        
    class hasUnit(NeXusField >> NeXusUnit):
        comment = 'NeXus Field has Unit relationship'
        
    class citesGroup(NXobject >> NeXusBaseClass):
        comment = 'NXobject cites base class relationship'
        
    class hasNXDLsource(NXobject >> str):
        comment = 'NXobject NXDL url as a string (xx change to url type)'
        
    class hasFieldName(NeXusField >> str):
        comment = 'NeXus field name of NeXus field. NeXus field names are unique only within the NeXus class in which they are defined.'
        
    ## relationships (generic)
    

onto.save()

print('Class\t\tComment\n')
for cls in onto.classes():
    print(cls, '\t', cls.comment[0])
print('\nProperty\t\tComment\n')
for prop in onto.object_properties():
    print(prop, '\t', prop.comment[0])
print('\nData\t\tComment\n')
for prop in onto.data_properties():
    print(prop, '\t', prop.comment[0]) 
    
#sync_reasoner() #doesn't work in JupyerHub - no JVM in container
#NeXusBaseClass.is_a

Class		Comment

NeXus_onto1.NeXus 	 NeXus concept bla bla
NeXus_onto1.NeXusField 	 NeXus field. Unique names are created by prepending the NeXus class name to the NeXus field name
NeXus_onto1.NXobject 	 NXobject
NeXus_onto1.NeXusBaseClass 	 NeXusBaseClass
NeXus_onto1.NeXusDefinition 	 NeXus Application Definition
NeXus_onto1.NeXusUnitCategory 	 NeXus Unit Category
NeXus_onto1.NeXusUnit 	 NeXus Unit

Property		Comment

NeXus_onto1.hasField 	 NXobject has NeXus field relationship
NeXus_onto1.citesGroup 	 NXobject cites base class relationship
NeXus_onto1.hasUnit 	 NeXus Field has Unit relationship

Data		Comment

NeXus_onto1.hasNXDLsource 	 NXobject NXDL url as a string (xx change to url type)
NeXus_onto1.hasFieldName 	 NeXus field name of NeXus field. NeXus field names are unique only within the NeXus class in which they are defined.
